In [ ]:
import pandas as pd
import http.client
import json
import requests
from bs4 import BeautifulSoup
import time
import concurrent.futures
import re

def find_linkedin_url(full_name, company_name):
    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({
        "q": f"{full_name} {company_name} linkedin"
    })
    headers = {
        'X-API-KEY': '46e6377865b21659da0a212efadbadf2129740f5',
        'Content-Type': 'application/json'
    }
    conn.request("POST", "/search", payload, headers)
    res = conn.getresponse()
    data = res.read().decode("utf-8")
    results = json.loads(data)
    if not results:
        return None
    if not results['organic']:
        return None
    website = results['organic'][0]['link']
    return website

def extract_linkedin_person_id(linkedin_url):
    if not linkedin_url:
        return None
    match = re.search(r'/in/([^/?#]+)', linkedin_url)
    return match.group(1) if match else None

def get_linkedin_id(full_name, company_name):
    linkedin_url = find_linkedin_url(full_name, company_name)
    if not linkedin_url:
        return None
    linkedin_id = extract_linkedin_person_id(linkedin_url)
    return linkedin_id

def process_company(full_name, company_name):
    linkedin_id = get_linkedin_id(full_name, company_name)
    return full_name, linkedin_id

df = pd.read_csv('experimentillo.csv')

# Create a full names Series using apply
df['full_name'] = df.apply(lambda row: f"{row['First Name']} {row['Last Name']}", axis=1)

# Pass to executor.map

# Using ThreadPoolExecutor to run 10 concurrent rows at a time
with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:
    results = list(executor.map(process_company, df['full_name'], df['Domain']))

# Convert the results into a DataFrame
results_df = pd.DataFrame(results, columns=['full_name', 'linkedin_id'])

# Merge the new DataFrame with the original one
df = df.merge(results_df, on='full_name')

df.to_csv('person_ids.csv')

In [ ]:
df = pd.read_csv('person_ids.csv')